In [3]:
import os
import pandas as pd
from openpyxl import load_workbook 
import win32com.client as win32
import sys
from pathlib import Path
import re
import glob

In [ ]:
def merge_mailaddress(df, address_data, left_on='Requested For', right_on='Name'):
    """
    メインデータフレームとアドレステーブルを結合
    
    Parameters:
    df: メインのデータフレーム
    address_data: アドレステーブル（Name, AddressName列を含む）
    left_on: 結合キー（メインDF側）
    right_on: 結合キー（アドレステーブル側）
    """
    try:
        # 最初に列名の前後の空白を除去
        print(f"  📋 元の列名: {list(df.columns)}")
        df.columns = df.columns.str.strip()
        print(f"  📋 修正後列名: {list(df.columns)}")
        
        # 結合キーが存在するかチェック
        if left_on not in df.columns:
            # Requested Forの候補を探す
            requested_for_candidates = [
                col for col in df.columns 
                if isinstance(col, str) and 'requested' in col.lower() and 'for' in col.lower()
            ]
            
            if requested_for_candidates:
                actual_key = requested_for_candidates[0]
                print(f"🔧 列名修正: '{left_on}' → '{actual_key}'")
                left_on = actual_key
            else:
                print(f"❌ 結合キー '{left_on}' が見つかりません")
                print(f"   利用可能な列: {list(df.columns)}")
                return df
        
        # 左結合でアドレス情報を追加
        merged_df = df.merge(address_data, left_on=left_on, right_on=right_on, how='left')
        
        # 結合結果の確認
        matched_count = merged_df['AddressName'].notna().sum()
        total_count = len(merged_df)
        print(f"アドレス結合結果: {matched_count}/{total_count} 件マッチ")
        
        # マッチしなかった名前を表示（NaNを除外）
        unmatched_mask = merged_df['AddressName'].isna() & merged_df[left_on].notna()
        unmatched = merged_df[unmatched_mask][left_on].unique()
        if len(unmatched) > 0:
            valid_unmatched = [name for name in unmatched if pd.notna(name)]
            if valid_unmatched:
                print(f"アドレス未登録: {valid_unmatched}")
        
        return merged_df
    
    except Exception as e:
        print(f"アドレス結合エラー: {e}")
        print(f"デバッグ情報:")
        print(f"  DataFrame列: {list(df.columns)}")
        print(f"  結合キー: {left_on}")
        return df


def parse_hyperlink_formula(formula):
    """
    HYPERLINK関数からURLと表示テキストを抽出
    """
    if not formula or not formula.startswith('=HYPERLINK'):
        return None, None
    
    pattern = r'=HYPERLINK\("([^"]+)"\s*,\s*"([^"]+)"\)'
    match = re.match(pattern, formula)
    
    if match:
        url = match.group(1)
        display_text = match.group(2)
        return url, display_text
    
    return None, None

def filter_dataframe(df):
    """
    データフレームから空の行や意味のないデータを含む行を自動除外
    
    Parameters:
    df: pandas DataFrame
    """
    original_len = len(df)
    filtered_df = df.copy()
    
    # 1. 完全に空の行を除外
    filtered_df = filtered_df.dropna(how='all')
    empty_rows_removed = original_len - len(filtered_df)
    if empty_rows_removed > 0:
        print(f"完全に空の行を {empty_rows_removed} 行削除")
    
    # 2. 主要列（Name, Document Name等）がすべて空の行を除外
    key_columns = []
    for col in ['Name', 'Document Name', 'Document Title', 'Requested For', 'Requested By']:
        if col in filtered_df.columns:
            key_columns.append(col)
    
    if key_columns:
        # 主要列がすべてNaNまたは空文字の行を除外
        mask = filtered_df[key_columns].notna().any(axis=1) & \
               (filtered_df[key_columns].astype(str) != '').any(axis=1)
        before_key_filter = len(filtered_df)
        filtered_df = filtered_df[mask]
        key_filter_removed = before_key_filter - len(filtered_df)
        if key_filter_removed > 0:
            print(f"主要データが空の行を {key_filter_removed} 行削除")
    
    # 3. テストデータや無効なデータを除外
    test_patterns = ['test', 'テスト', 'sample', 'サンプル', 'dummy', 'ダミー']
    
    for col in filtered_df.columns:
        if filtered_df[col].dtype == 'object':  # 文字列列のみ
            before_test_filter = len(filtered_df)
            mask = ~filtered_df[col].astype(str).str.lower().str.contains(
                '|'.join(test_patterns), case=False, na=False, regex=True
            )
            filtered_df = filtered_df[mask]
            test_filter_removed = before_test_filter - len(filtered_df)
            if test_filter_removed > 0:
                print(f"テストデータを {test_filter_removed} 行削除 (列: {col})")
                break  # 一度に大量削除を避けるため、最初の一致で停止
    
    # 4. 重複行を除外（すべての列が同じ場合）
    before_duplicate_filter = len(filtered_df)
    filtered_df = filtered_df.drop_duplicates()
    duplicate_removed = before_duplicate_filter - len(filtered_df)
    if duplicate_removed > 0:
        print(f"重複行を {duplicate_removed} 行削除")
    
    print(f"📊 フィルタリング結果: {original_len}行 → {len(filtered_df)}行 (削除: {original_len - len(filtered_df)}行)")
    return filtered_df


def extract_recipients_from_excel(excel_paths, address_data, sheet_name=None, column_name='AddressName'):
    """
    複数のExcelファイルからRequested For列のユニークなメールアドレスを取得
    
    Parameters:
    excel_paths: list of str - Excelファイルパスのリスト
    address_data: DataFrame - アドレステーブル（Name, AddressName列）
    sheet_name: str - シート名（Noneの場合は最初のシート）
    column_name: str - メールアドレス列名
    
    Returns:
    list: ユニークなメールアドレスのリスト
    """
    try:
        recipients_addresses = set()
        processed_files = 0
        
        for excel_path in excel_paths:
            try:
                print(f"📄 アドレス抽出中: {excel_path}")
                
                # Excelファイル読み込み
                if sheet_name is None:
                    excel_data = pd.read_excel(excel_path, sheet_name=None)
                    first_sheet_name = list(excel_data.keys())[0]
                    df = excel_data[first_sheet_name]
                    print(f"  使用シート: {first_sheet_name}")
                else:
                    df = pd.read_excel(excel_path, sheet_name=sheet_name)
                    print(f"  使用シート: {sheet_name}")
                
                # アドレステーブルと結合
                df_merged = merge_mailaddress(df, address_data)
                
                # メールアドレス抽出
                if column_name in df_merged.columns:
                    # NaN、空文字、無効なアドレスを除外
                    valid_addresses = df_merged[column_name].dropna()
                    valid_addresses = valid_addresses.astype(str).str.strip()
                    valid_addresses = valid_addresses[valid_addresses != '']
                    
                    # メールアドレス形式をチェック（@を含む）
                    valid_addresses = valid_addresses[valid_addresses.str.contains('@', na=False)]
                    
                    # ユニークな値を取得してセットに追加
                    unique_addresses = valid_addresses.unique().tolist()
                    recipients_addresses.update(unique_addresses)
                    
                    print(f"  抽出件数: {len(unique_addresses)} 件")
                    if unique_addresses:
                        print(f"  抽出例: {unique_addresses[0]}")
                
                else:
                    print(f"  ⚠️ 警告: '{column_name}'列が見つかりません")
                    print(f"     利用可能な列: {list(df_merged.columns)}")
                
                processed_files += 1
                
            except Exception as file_error:
                print(f"  ❌ ファイル処理エラー: {file_error}")
                continue
        
        # 結果の整理
        final_addresses = list(recipients_addresses)
        final_addresses.sort()  # アルファベット順にソート
        
        print(f"\n📧 最終結果:")
        print(f"処理ファイル数: {processed_files}/{len(excel_paths)}")
        print(f"ユニークアドレス数: {len(final_addresses)}")
        
        if final_addresses:
            print("抽出されたアドレス:")
            for addr in final_addresses:
                print(f"  - {addr}")
        else:
            print("⚠️ 有効なメールアドレスが見つかりませんでした")
        
        return final_addresses
    
    except Exception as e:
        print(f"❌ 受信者抽出エラー: {e}")
        import traceback
        traceback.print_exc()
        return []


def extract_table_with_formula_links(excel_path, sheet_name=None):
    """
    HYPERLINK関数を含む表をHTML形式で抽出（自動フィルタリング対応）
    """
    # pandasでデータ読み込み
    if sheet_name is None:
        excel_data = pd.read_excel(excel_path, sheet_name=None)
        first_sheet_name = list(excel_data.keys())[0]
        df = excel_data[first_sheet_name]
        used_sheet_name = first_sheet_name
    else:
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
        used_sheet_name = sheet_name
    
    # データフィルタリング（自動）
    df_filtered = filter_dataframe(df)
    
    # openpyxlでセル情報取得
    wb = load_workbook(excel_path, data_only=False)
    if sheet_name is None:
        ws = wb.worksheets[0]
    elif sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
    else:
        ws = wb.worksheets[0]
    
    # HTMLテーブル作成
    html_rows = []
    
    # ヘッダー行
    headers = df_filtered.columns.tolist()
    header_row = "<tr>" + "".join([f"<th style='border: 1px solid #ddd; padding: 6px; background-color: #f2f2f2; font-weight: bold; font-size: 12px;'>{header}</th>" for header in headers]) + "</tr>"
    html_rows.append(header_row)
    
    # データ行
    for index, row in df_filtered.iterrows():
        html_cells = []
        for col_idx, (col_name, cell_value) in enumerate(row.items()):
            # 元のDataFrameでの行番号を取得
            original_index = df_filtered.index[df_filtered.index == index][0]
            excel_row = original_index + 2  # ヘッダー分+1
            excel_col = col_idx + 1
            
            # セル取得
            cell = ws.cell(row=excel_row, column=excel_col)
            
            # 数式をチェック
            formula = cell.value if isinstance(cell.value, str) and cell.value.startswith('=') else None
            
            # HYPERLINK関数の処理
            if formula and 'HYPERLINK' in formula:
                url, display_text = parse_hyperlink_formula(formula)
                if url and display_text:
                    cell_html = f"<a href='{url}' style='color: blue; text-decoration: underline;' target='_blank'>{display_text}</a>"
                else:
                    cell_html = str(cell_value) if cell_value is not None else ""
            
            # 通常のハイパーリンクの処理
            elif cell.hyperlink:
                link_url = cell.hyperlink.target
                if link_url and link_url.startswith(('http://', 'https://')):
                    cell_html = f"<a href='{link_url}' style='color: blue; text-decoration: underline;' target='_blank'>{cell_value}</a>"
                else:
                    cell_html = str(cell_value) if cell_value is not None else ""
            
            # リンクなしの場合
            else:
                cell_html = str(cell_value) if cell_value is not None else ""
            
            html_cells.append(f"<td style='border: 1px solid #ddd; padding: 6px; vertical-align: top; font-size: 11px;'>{cell_html}</td>")
        
        html_rows.append("<tr>" + "".join(html_cells) + "</tr>")
    
    # ファイル名をタイトルとして追加
    file_name = Path(excel_path).name
    html_table = f"""
    <div style='margin: 15px 0;'>
        <h3 style='color: #333; margin-bottom: 5px; font-size: 14px;'>📄 {file_name} ({used_sheet_name})</h3>
        <table style='border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; margin-bottom: 10px;'>
            {"".join(html_rows)}
        </table>
    </div>
    """
    
    return html_table

def send_multi_excel_email(excel_files, recipients=None, subject="複数データ送付", message=""):
    """
    複数のExcelファイルを処理してメール送信
    
    Parameters:
    excel_files: list of str - Excelファイルパスのリスト
    recipients: list of str - 受信者リスト（Noneの場合は最初のファイルから自動抽出）
    subject: str - メール件名
    message: str - 追加メッセージ
    """
    
    try:
        # ファイル存在チェック
        valid_files = []
        for file_path in excel_files:
            if Path(file_path).exists():
                valid_files.append(file_path)
                print(f"✅ ファイル確認: {Path(file_path).name}")
            else:
                print(f"❌ ファイルが見つかりません: {file_path}")
        
        if not valid_files:
            raise FileNotFoundError("有効なExcelファイルがありません")
        
        # 受信者を自動抽出（指定されていない場合）
        if recipients is None:
            print("受信者を最初のファイルから抽出中...")
            recipients = extract_recipients_from_excel(valid_files[0])
            if not recipients:
                raise ValueError("受信者が抽出できませんでした")
        
        print(f"受信者: {'; '.join(recipients)}")
        
        # 各ファイルからHTMLテーブルを生成
        html_tables = []
        for file_path in valid_files:
            print(f"\n📄 処理中: {Path(file_path).name}")
            html_table = extract_table_with_formula_links(file_path)
            html_tables.append(html_table)
        
        print("Outlookに接続中...")
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        
        # 受信者設定
        mail.To = "; ".join(recipients)
        mail.Subject = subject
        
        # HTMLメール本文作成
        html_body = f"""
        <html>
        <head>
            <meta charset="UTF-8">
            <style>
                body {{ font-family: Arial, sans-serif; line-height: 1.4; }}
                .header {{ background-color: #f8f9fa; padding: 15px; border-left: 4px solid #007acc; margin: 10px 0; }}
                .summary {{ background-color: #e7f3ff; padding: 10px; margin: 10px 0; border-radius: 5px; }}
                a {{ color: #0066cc; }}
                a:hover {{ color: #004499; }}
                h3 {{ color: #333; border-bottom: 2px solid #007acc; padding-bottom: 5px; }}
            </style>
        </head>
        <body>
            <div class="header">
                <strong>📊 データ送付のお知らせ</strong><br>
                {message if message else "添付ファイルのデータをお送りします。"}
            </div>
            
            <div class="summary">
                <strong>📋 送付ファイル一覧:</strong>
                <ul>
                    {"".join([f"<li>{Path(f).name}</li>" for f in valid_files])}
                </ul>
            </div>
            
            <h3>📈 データ詳細</h3>
            {"".join(html_tables)}
            
            <div style="margin-top: 20px; padding: 10px; background-color: #f8f9fa; border-radius: 5px;">
                <em>💾 詳細情報や元データは添付のExcelファイルをご参照ください。</em>
            </div>
        </body>
        </html>
        """
        
        mail.HTMLBody = html_body
        
        # 複数ファイルを添付
        print("ファイルを添付中...")
        for file_path in valid_files:
            mail.Attachments.Add(str(Path(file_path).absolute()))
            print(f"  ✅ 添付: {Path(file_path).name}")
        
        print("メールを送信中...")
        mail.Send()
        print(f"✅ メール送信完了: {'; '.join(recipients)}")
        
        return True
        
    except Exception as e:
        print(f"❌ エラーが発生しました: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

def main_multi_excel():
    """
    複数Excel対応版のメイン実行
    """
    print("=== 複数Excel自動メール送信ツール ===\n")
    
    # 設定値（ここを編集してください）
    EXCEL_FOLDER = "Excel folder path" # ここにExcelファイルがダウンロードされたフォルダーパスを入れる
    
    # フォルダ内の全Excelファイルを自動検出
    EXCEL_FILES = glob.glob(os.path.join(EXCEL_FOLDER, "*.xlsx"))
    ADDRESS_BOOK_PATH = "Address path to PSJ member list" # PSJのメンバーの名前、アドレスが保存されたエクセルファイルのパスを入れる。
    
    EMAIL_SUBJECT = "データ送付テスト (複数ファイル)" # 必要に応じて修正
    MESSAGE = "いつもお世話になっております。複数のデータファイルをお送りします。" #必要に応じて修正
    
    # アドレスブックを読み込み
    try:
        print(f"📒 アドレスブック読み込み中: {Path(ADDRESS_BOOK_PATH).name}")
        address_data = pd.read_excel(ADDRESS_BOOK_PATH)
        address_data.columns = ["AddressName", "Name"]
        print(f"  アドレスブック列: {list(address_data.columns)}")
        print(f"  アドレス件数: {len(address_data)}件")
        
        # アドレスブックに適切な列があるかチェック
        if 'Name' not in address_data.columns:
            print("❌ アドレスブックに'Name'列がありません")
            return
        if 'AddressName' not in address_data.columns:
            print("❌ アドレスブックに'AddressName'列がありません")
            print(f"   利用可能な列: {list(address_data.columns)}")
            return
            
    except Exception as e:
        print(f"❌ アドレスブック読み込みエラー: {e}")
        return
    
    # 受信者の取得
    try:
        recipients = extract_recipients_from_excel(
            excel_paths=EXCEL_FILES,
            address_data=address_data,  # DataFrameとして渡す
            sheet_name=None,
            column_name='AddressName'  
        )
                    
    except Exception as e:
        print(f"❌ 受信者抽出エラー: {e}")
        return
    
    if not recipients:
        print("❌ 有効な受信者が見つかりませんでした。処理を終了します。")
        return
    
    print("設定確認:")
    print(f"処理ファイル数: {len(EXCEL_FILES)}")
    for i, file_path in enumerate(EXCEL_FILES, 1):
        print(f"  {i}. {Path(file_path).name}")
    print(f"件名: {EMAIL_SUBJECT}")
    print()
    print("送付先アドレス:")
    for addr in recipients:
        print(f"  - {addr}")
        
    confirm = input("この設定でメールを送信しますか？ (y/N): ")
    if confirm.lower() != 'y':
        print("送信をキャンセルしました。")
        return
    
    # 修正: exclude_patterns引数を削除
    success = send_multi_excel_email(
        excel_files=EXCEL_FILES,
        recipients=recipients,
        subject=EMAIL_SUBJECT,
        message=MESSAGE
    )
    
    if success:
        print("\n🎉 処理が正常に完了しました！")
    else:
        print("\n💀 処理中にエラーが発生しました。")


In [14]:
main_multi_excel()

=== 複数Excel自動メール送信ツール ===

📒 アドレスブック読み込み中: 202506_PSJ_memberlist.xlsx
  アドレスブック列: ['AddressName', 'Name']
  アドレス件数: 294件
📄 アドレス抽出中: C:\Users\GPGTT\OneDrive - Bayer\Desktop\Onoda-san\excel_data\Controlled Print_JP01_In Recall.xlsx
  使用シート: Sheet0
  📋 元の列名: ['Name', 'Created Date', 'Reason for copy', 'Requested For', 'Requested By', 'Document Name', 'Document Title', 'Version', 'Name.1']
  📋 修正後列名: ['Name', 'Created Date', 'Reason for copy', 'Requested For', 'Requested By', 'Document Name', 'Document Title', 'Version', 'Name.1']
アドレス結合結果: 9/19 件マッチ
  ⚠️ 警告: 'MailAddress'列が見つかりません
     利用可能な列: ['Name_x', 'Created Date', 'Reason for copy', 'Requested For', 'Requested By', 'Document Name', 'Document Title', 'Version', 'Name.1', 'AddressName', 'Name_y']
📄 アドレス抽出中: C:\Users\GPGTT\OneDrive - Bayer\Desktop\Onoda-san\excel_data\Legacy Controlled Print Trace_In Recall.xlsx
  使用シート: Sheet0
  📋 元の列名: ['Name', 'Created Date', ' Reason for copy', 'Created By', ' Requested For', 'Requested By', 'Legac